In [1]:
import pandas as pd
from tkinter import *
from tkinter import filedialog
from tkinter import ttk
import tkinter as tk
from sklearn.utils import shuffle
import os

root = tk.Tk()
root.title("Foreign To Local Dataset Converter")
root.geometry("640x640+600+200")

heading = tk.Label(root, text="\nForeign Dataset to SG Context Mapper\n", font=("Helvetica",20,"bold"))
heading.pack()

addAddressLAbel = tk.Label(root, text="Would you like to map addresses?", font=("Helvetica",10,"bold")).pack()

#Declaring Variables For creation of New column (Postal code, Address)
postalCodeArray = []
addressArray = []

#Importing Dataset

#If User selects that they want address to be mapped (Region & Address)
def includeAddress():
    global booleanVar
    booleanVar = "Yes"
    includeMsg = Label(root, text= "You have selected Yes",fg = "green").pack()
    importAddressBtn.pack()
    
#If User selects that they do not want address to be mapped (Region Only)
def excludeAddress():
    global booleanVar
    booleanVar = "No"
    excludeMsg = Label(root, text= "You have selected No",fg = "green").pack()
    importTownBtnNo.pack()

#Importing Foreign Dataset
def importForeignDataset():
    global ForeignDataset
    ForeignDataset = filedialog.askopenfilename( filetypes = (("All Files", "*.*"),("CSV Files","*.csv"),("XLSX Files","*.xlsx") ), title = "Choose Foreign Dataset For Mapping" )
    
    if ForeignDataset == "" :
        foreignDatasetLabelFail = Label(root, text= "Please Import a Foreign Dataset", fg="red").pack()
    else :
        foreignFilename = os.path.split(ForeignDataset)[1]
        foreignDatasetLabel = Label(root, text= "Successfully Imported Foreign Dataset : [ " + foreignFilename +" ]", fg="green").pack()
        columnNameInputLabel = Label(root, text="Enter column name: ", font=("arial",10,"bold")).place(x=100,y=410)
        
        global columnName
        columnName = StringVar()
        columnNameInputField = Entry(root, textvariable=columnName, width=25, bg="lightgreen").place(x=260,y=410)
        root.bind('<Return>', showBtn)

def showBtn(event):
    global columnName2
    columnName2 = columnName.get()
    
    if booleanVar == "Yes":
        runProgramBtn.place(x=278,y=445)
    else:
        runProgramBtnNo.place(x=278,y=445)

#Importing Address Dataset
def importAddressDataset():
    global addressPool
    addressPool = filedialog.askopenfilename( filetypes = (("All Files", "*.*"),("CSV Files","*.csv"),("XLSX Files","*.xlsx") ), title = "Choose Singapore Addresses Dataset" )
    
    if addressPool == "":
        addressPoolLabelFail =  Label(root, text= "Please Import Singapore Addresses Dataset", fg="red").pack()
    else:
        addressFilename = os.path.split(addressPool)[1]
        addressDatasetLabel = Label(root, text= "Successfully Imported Addresses Dataset : [ " + addressFilename +" ]", fg="green")
        addressDatasetLabel.pack()
        if booleanVar == "Yes":
            importTownBtn.pack()
        else:
            importTownBtnNo.pack()

#Importing Town Dataset
def importTownDataset():
    global SingaporeTown
    SingaporeTown = filedialog.askopenfilename( filetypes = (("All Files", "*.*"),("CSV Files","*.csv"),("XLSX Files","*.xlsx") ), title = "Choose Singapore Town Dataset" )
    
    if SingaporeTown == "":
        SingaporeTownFailLabel = Label(root, text= "Please Import Singapore Region Dataset", fg="red").pack()
    else:
        townFilename = os.path.split(SingaporeTown)[1]
        townDatasetLabel = Label(root, text= "Successfully Imported Town Dataset : [ " + townFilename +" ]", fg="green")
        townDatasetLabel.pack()
        if booleanVar == "Yes":
            importForeignBtn.pack()
        else:
            importForeignBtnNo.pack()

#Creating the columns Postal Code and Address (Retrieve from dataset)            
def getAddress():
    df_addressPoolFiltered = df_addressPool[df_addressPool['Region'] == SGcode2]
    df_shuffledAddresses = shuffle(df_addressPoolFiltered)

    newPostalCode = df_shuffledAddresses["Postal Code"].iloc[0]
    newAddress = df_shuffledAddresses["Address"].iloc[0]
    
    postalCodeArray.append(newPostalCode)
    addressArray.append(newAddress)

# Store excel/csv files as dataframe & Run the whole program without adding Addresses & Postal Code
def runProgramNo():
    runningMessage = Label(root, text= "Code Running...",fg = "green").place(x=270, y=475)
    
    print("Running")
    print(columnName2)
    
    checkFormat = ForeignDataset[-3:]
    
    global df_ForeignDataset
    
    if checkFormat == "csv":
        df_ForeignDataset = pd.read_csv(ForeignDataset)
    if checkFormat == "lsx":
        df_ForeignDataset = pd.read_excel(ForeignDataset)
    
    global df_SingaporeTown
    df_SingaporeTown = pd.read_excel(SingaporeTown)
    
    ForeignCode = df_ForeignDataset[columnName2].unique()
    ForeignCodeList = ForeignCode.tolist()
    SGTowns = df_SingaporeTown.Town.unique()
    SGTownsList = SGTowns.tolist()
    
    
    for ind,row in df_ForeignDataset.iterrows():
        currentForeignCode = row[columnName2]
        currentForeignCode_index = ForeignCodeList.index(currentForeignCode)
        SGcode = df_SingaporeTown["Town"][currentForeignCode_index]
        df_ForeignDataset[columnName2][ind] = df_ForeignDataset[columnName2][ind].replace(currentForeignCode, SGcode)
        
        global SGcode2
        SGcode2 = SGcode #(Works just need to make sure dataset of addresses and town matches)
        

    df_ForeignDataset.rename(columns={columnName2: 'Region'}, inplace=True)
    df_ForeignDataset.to_csv('UpdatedDataset.csv')
    
    completedMessage = Label(root, text= "Mapping Completed!",fg = "red").place(x=257,y=500)
    

# Store excel/csv files as dataframe & Run the whole program & adding Addresses & Postal Code    
def runProgram():
    
    runningMessage = Label(root, text= "Code Running...",fg = "green").place(x=270, y=475)

    print("Running")
    print(columnName2)
    
    checkFormat = ForeignDataset[-3:]
    
    global df_ForeignDataset
    
    if checkFormat == "csv":
        df_ForeignDataset = pd.read_csv(ForeignDataset)
    if checkFormat == "lsx":
        df_ForeignDataset = pd.read_excel(ForeignDataset)
    
    global df_addressPool
    df_addressPool = pd.read_excel(addressPool, dtype={'Region': str, 'Street Name': str, 'Address': str, 'Postal Code': str })
    global df_SingaporeTown
    df_SingaporeTown = pd.read_excel(SingaporeTown)
    
    ForeignCode = df_ForeignDataset[columnName2].unique()
    ForeignCodeList = ForeignCode.tolist()
    SGTowns = df_SingaporeTown.Town.unique()
    SGTownsList = SGTowns.tolist()
    
    
    for ind,row in df_ForeignDataset.iterrows():
        currentForeignCode = row[columnName2]
        currentForeignCode_index = ForeignCodeList.index(currentForeignCode)
        SGcode = df_SingaporeTown["Town"][currentForeignCode_index]
        df_ForeignDataset[columnName2][ind] = df_ForeignDataset[columnName2][ind].replace(currentForeignCode, SGcode)
        
        df_addressPoolFiltered = df_addressPool[df_addressPool['Region'] == SGcode]
        df_shuffledAddresses = shuffle(df_addressPoolFiltered)
        
        global SGcode2
        SGcode2 = SGcode #(Works just need to make sure dataset of addresses and town matches)
        getAddress()
        
    
    df_ForeignDataset['Postal Code'] = postalCodeArray
    df_ForeignDataset['Address'] = addressArray

    df_ForeignDataset.rename(columns={columnName2: 'Region'}, inplace=True)
    df_ForeignDataset.to_csv('UpdatedDataset.csv')
    
    completedMessage = Label(root, text= "Mapping Completed!",fg = "red").place(x=257,y=500)
    
#Putting Elements on Window
importForeignBtn = ttk.Button(root, text="3. Import Foreign Dataset", command=importForeignDataset)
importAddressBtn = ttk.Button(root, text="1. Import Address Dataset", command=importAddressDataset)
importTownBtn = ttk.Button(root, text="2. Import Singapore Region Dataset", command=importTownDataset)
runProgramBtn = ttk.Button(root, text="4. Run", command=runProgram)


importTownBtnNo = ttk.Button(root, text="1. Import Singapore Region Dataset", command=importTownDataset)
importForeignBtnNo = ttk.Button(root, text="2. Import Foreign Dataset", command=importForeignDataset)
runProgramBtnNo = ttk.Button(root, text="3. Run", command=runProgramNo)

includeAddressBtn = ttk.Button(root, text="Yes", command=includeAddress).pack()
excludeAddressBtn = ttk.Button(root, text="No", command=excludeAddress).pack()

#importAddressBtn.pack()

root.mainloop()
#root.withdraw()



Running
State


C:\Users\L31206\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
